In [1]:
import pandas as pd
# Import Library
import random
import torch, torchtext
from torchtext import data 


# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [2]:
!pip install googletrans==3.1.0a0
import random
import googletrans
from googletrans import Translator

     |████████████████████████████████| 61kB 2.8MB/s 
     |████████████████████████████████| 983kB 6.8MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 71kB 4.1MB/s 
     |████████████████████████████████| 61kB 6.0MB/s 
     |████████████████████████████████| 102kB 7.5MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp36-none-any.whl size=16369 sha256=36e452b9039d38586fe780c0f00fae91f745ecfd77b74eb978aa071717cc77bf
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=c2c2fc0e8d071cf66807642a0445810b496329ce20b7fdaa03de984879159e43
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletrans contextvars


In [3]:
!wget http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip

--2020-12-10 14:11:05--  http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip [following]
--2020-12-10 14:11:06--  https://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6372817 (6.1M) [application/zip]
Saving to: ‘stanfordSentimentTreebank.zip’

stanfordSentimentTr 100%[===================>]   6.08M   689KB/s    in 9.3s    

2020-12-10 14:11:16 (672 KB/s) - ‘stanfordSentimentTreebank.zip’ saved [6372817/6372817]



In [4]:
!unzip stanfordSentimentTreebank.zip

Archive:  stanfordSentimentTreebank.zip
   creating: stanfordSentimentTreebank/
  inflating: stanfordSentimentTreebank/datasetSentences.txt  
   creating: __MACOSX/
   creating: __MACOSX/stanfordSentimentTreebank/
  inflating: __MACOSX/stanfordSentimentTreebank/._datasetSentences.txt  
  inflating: stanfordSentimentTreebank/datasetSplit.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._datasetSplit.txt  
  inflating: stanfordSentimentTreebank/dictionary.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._dictionary.txt  
  inflating: stanfordSentimentTreebank/original_rt_snippets.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._original_rt_snippets.txt  
  inflating: stanfordSentimentTreebank/README.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._README.txt  
  inflating: stanfordSentimentTreebank/sentiment_labels.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._sentiment_labels.txt  
  inflating: stanfordSentimentTreebank/SOStr.txt  
  inflating: stanfo

Preprocess the dataset and derive Train, Test and Validation data

In [5]:
import numpy as np
import unicodedata

root = "stanfordSentimentTreebank/"

sentences_file = open(root+"datasetSentences.txt")
sentences=[]
sentences_id = {}
for line in sentences_file:
	sentence = line.split("\t")[1].strip("\n")
	# sentence = unicode(line.split("\t")[1].strip("\n"),"ISO-8859-1")
	# sentence = sentence.encode("utf-8")
	# unicodedata.normalize('NFKD', sentence).encode('ascii','ignore')
	# print sentence
	sentences.append(sentence)
	sentences_id[sentence]=line.split("\t")[0]
sentences_file.close()

phrase_id_file = open(root+"dictionary.txt")
phrases={}
for line in phrase_id_file:
	line=line.split("|")
	phrases[line[0]]=line[1].strip("\n")
phrase_id_file.close()

phraseid_sentiment_file = open(root+"sentiment_labels.txt")
phrase_sentiments = {}
for line in phraseid_sentiment_file:
	line = line.split("|")
	phrase_sentiments[line[0]]=line[1].strip("\n")
phraseid_sentiment_file.close()

sentenceid_splitset_file = open(root+"datasetSplit.txt")
sentenceid_splitset = {}
for line in sentenceid_splitset_file:
	line = line.split(",")
	# print line
	sentenceid_splitset[line[0]]=line[1].strip("\n")
sentenceid_splitset_file.close()


def get_class(label):
	k=float(label)*5.0
	if k>=0 and k<=1:
		return "0"
	elif k>=1 and k<=2:
		return "1"
	elif k>=2 and k<=3:
		return "2"
	elif k>=3 and k<=4:
		return "3"
	elif k>=4 and k<=5:
		return "4"

sentence_labels_file = open("sentence_labels.txt","w")
train_file = open("train.txt","w")
valid_file = open("valid.txt","w")
test_file = open("test.txt","w")
train_file.write("Sentence\tScore\tLabel\n")
valid_file.write("Sentence\tScore\tLabel\n")
test_file.write("Sentence\tScore\tLabel\n")
for sentence in sentences:
	if sentence in phrases and sentence in sentences_id:
		sentence_id = sentences_id[sentence]
		split_set = sentenceid_splitset[sentence_id]
		phrase_id = phrases[sentence]
		label = phrase_sentiments[phrase_id]
		sentence_labels_file.write(sentence+"\t"+label+"\n")
		if split_set=="1":
			train_file.write(sentence+"\t"+label+"\t"+get_class(label)+"\n")
		elif split_set=="2":
			test_file.write(sentence+"\t"+label+"\t"+get_class(label)+"\n")
		elif split_set=="3":
			valid_file.write(sentence+"\t"+label+"\t"+get_class(label)+"\n")

		# print sentence, label
sentence_labels_file.close()
train_file.close()
valid_file.close()
test_file.close()

In [44]:
train_df = pd.read_csv('train.txt',sep='\t')
train_df.head()


,Sentence,Score,Label
0,The Rock is destined to be the 21st Century 's...,0.69444,3
1,The gorgeously elaborate continuation of `` Th...,0.83333,4
2,Singer\/composer Bryan Adams contributes a sle...,0.62500,3
3,You 'd think by now America would have had eno...,0.50000,2
4,Yet the act is still charming here .,0.72222,3


In [42]:
def random_deletion(sentence, p=0.5): 
    words = sentence.split()
    if len(words) == 1: # return if single word
        return ' '.join(words)
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return ' '.join([random.choice(words)])
    else:
        return ' '.join(remaining)

In [45]:
def random_swap(sentence, n=5): 
    sentence = sentence.split()
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return ' '.join(sentence)

In [46]:
translator = Translator()
def backTranslate(sentence):
  #print(Sentence)
  available_langs = list(googletrans.LANGUAGES.keys())
  trans_lang = random.choice(available_langs)
  translations = translator.translate(sentence, dest=trans_lang) 
  #print(translations)
  t_text = [t.text for t in translations]
  translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
  en_text = [t.text for t in translations_en_random]
  return ' '.join(en_text)

Augment every third data sample randomly. Either random deletion, or random swap or back translate is performed

In [47]:
df_len = train_df.shape[0]
augmentedList = []
# Augment every third review
for i in range(0,df_len,3):
  datadf = train_df.iloc[i]
  index = random.choice([1,2,3])
  if index == 1:
    result = random_deletion(datadf.Sentence)
    #augmentedList.append([result, datadf.Score, datadf.Label])
  elif index == 2:
    result = random_swap(datadf.Sentence)
    #augmentedList.append([result, datadf.Score, datadf.Label])
  elif index == 3:
    result = backTranslate([datadf.Sentence])
  augmentedList.append([result, datadf.Score, datadf.Label])
    



In [48]:
augmentedData = pd.DataFrame(augmentedList,columns=['Sentence','Score','Label'])
augmentedData.head()

,Sentence,Score,Label
0,The Rock is destined the be Conan 21st Century...,0.69444,3
1,You 'd by America had of eccentrics hearts gold,0.50000,2
2,Just the in involved this creating the layered...,0.87500,4
3,. Extreme Ops expectations exceeds `` '',0.73611,3
4,Dramas this .,0.80556,4


In [49]:
train_df_copy = train_df
train_df = pd.concat([train_df, augmentedData], ignore_index = True) 
train_df.reset_index() 
train_df.head()

,Sentence,Score,Label
0,The Rock is destined to be the 21st Century 's...,0.69444,3
1,The gorgeously elaborate continuation of `` Th...,0.83333,4
2,Singer\/composer Bryan Adams contributes a sle...,0.62500,3
3,You 'd think by now America would have had eno...,0.50000,2
4,Yet the act is still charming here .,0.72222,3


In [50]:
train_df.shape

(10828, 3)

Write the augment data to new file

In [51]:
train_df.to_csv('train.csv')

In [70]:

Sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)
def transformData(file,sep='\t'):
  df = pd.read_csv(file,sep=sep)
  # shuffle the DataFrame rows 
  df = df.sample(frac = 1).reset_index(drop=True)
  fields = [('Sentence', Sentence),('Label',Label)]
  example = [data.Example.fromlist([df.Sentence[i],df.Label[i]], fields) for i in range(df.shape[0])] 
  dataset = data.Dataset(example, fields)
  return dataset

In [71]:
train = transformData('train.txt')
#train = transformData('train.csv',sep=',')
valid = transformData('valid.txt')
test = transformData('test.txt')

In [73]:
vars(train.examples[11])

{'Label': 2,
 'Sentence': ['Veers',
  'uncomfortably',
  'close',
  'to',
  'pro',
  '-',
  'Serb',
  'propaganda',
  '.']}

In [74]:
Sentence.build_vocab(train)
Label.build_vocab(train)

In [75]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  16524
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 7637), (',', 6706), ('the', 5714), ('of', 4170), ('and', 4152), ('a', 4147), ('to', 2844), ('-', 2566), ('is', 2403), ("'s", 2354)]
Labels :  defaultdict(<function _default_unk_index at 0x7fb707d101e0>, {3: 0, 1: 1, 2: 2, 4: 3, 0: 4})


In [76]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [77]:
train_iterator, valid_iterator,test_iterator = data.BucketIterator.splits((train, valid,test), batch_size = 32, 
                                                            sort_key = lambda x: len(x.Sentence),
                                                            sort_within_batch=True, device = device)

In [78]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Sentence.vocab.stoi, tokens)

In [79]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.dropout = nn.Dropout(0.5)
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
        embedded = self.dropout(embedded)
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        #dense_outputs = self.dropout(dense_outputs)
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [80]:
len(Sentence.vocab)

16524

In [81]:
# Define hyperparameters
size_of_vocab = len(Sentence.vocab)
embedding_dim = 300
num_hidden_nodes = 512
num_output_nodes = len(Label.vocab)
num_layers = 2
dropout = 0.5

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [82]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(16524, 300)
  (dropout): Dropout(p=0.5, inplace=False)
  (encoder): LSTM(300, 512, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=512, out_features=5, bias=True)
)
The model has 8,728,085 trainable parameters


In [83]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [84]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        Sentence, Sentence_lengths = batch.Sentence   
        
        # convert to 1D tensor
        predictions = model(Sentence, Sentence_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.Label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.Label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [85]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            Sentence, Sentence_lengths = batch.Sentence
            
            # convert to 1d tensor
            predictions = model(Sentence, Sentence_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.Label)
            acc = binary_accuracy(predictions, batch.Label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [86]:
N_EPOCHS = 15
best_valid_loss = float('inf')
batch_size = 32
f = open("results_lstm.txt",'a')
f.write(f'Start Training the Model\n\n')
for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')

    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')
    f.write("batch_size : "+str(batch_size)+"  nb_epoch : "+str(epoch)+"   "+"optimizer : adam"+"\n")
    f.write(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    f.write(f'\t Vl. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.578 | Train Acc: 27.03%
	 Val. Loss: 1.576 |  Val. Acc: 26.83% 

	Train Loss: 1.567 | Train Acc: 29.66%
	 Val. Loss: 1.569 |  Val. Acc: 29.84% 

	Train Loss: 1.553 | Train Acc: 32.57%
	 Val. Loss: 1.543 |  Val. Acc: 33.89% 

	Train Loss: 1.515 | Train Acc: 36.72%
	 Val. Loss: 1.539 |  Val. Acc: 34.14% 

	Train Loss: 1.477 | Train Acc: 41.44%
	 Val. Loss: 1.542 |  Val. Acc: 34.72% 

	Train Loss: 1.441 | Train Acc: 45.31%
	 Val. Loss: 1.522 |  Val. Acc: 36.41% 

	Train Loss: 1.401 | Train Acc: 49.70%
	 Val. Loss: 1.529 |  Val. Acc: 35.03% 

	Train Loss: 1.376 | Train Acc: 52.26%
	 Val. Loss: 1.531 |  Val. Acc: 35.76% 

	Train Loss: 1.340 | Train Acc: 56.02%
	 Val. Loss: 1.536 |  Val. Acc: 35.07% 

	Train Loss: 1.311 | Train Acc: 59.36%
	 Val. Loss: 1.553 |  Val. Acc: 33.06% 

	Train Loss: 1.294 | Train Acc: 60.92%
	 Val. Loss: 1.545 |  Val. Acc: 34.57% 

	Train Loss: 1.268 | Train Acc: 63.36%
	 Val. Loss: 1.529 |  Val. Acc: 35.20% 

	Train Loss: 1.253 | Train Acc: 64.99%
	

In [87]:

test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'\t Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}% \n')
f.write(f'\n\nTesting the Model')
f.write(f'\n\n\n\t Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}% \n')
f.close()

	 Test Loss: 1.514 |  Test Acc: 37.89% 

